In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import time
import os
from PIL import ImageFile

In [ ]:
#Biar kalau ada gambar yang corrupt, tetap jalan
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
#Parameters
data_directory = 'gen3_dataset'
batch_size = 32
learning_rate = 0.001
epochs = 30
img_size = 224


In [ ]:
def train_model():
    #Pakai GPU kalo ada
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Datanya diubah-ubah agar model gak nginget gambar doang soalnya jumlah image dikit
    transform_train = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(img_size),
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    transform_val = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    full_data = datasets.ImageFolder(data_directory, transform=transform_train)
    #Train Val Split 0.8:0.2
    train_size = int(0.8 * len(full_data))
    val_size = len(full_data) - train_size
    train_data, val_data = random_split(full_data, [train_size, val_size])

    val_data.dataset.transform = transform_val
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2)

    class_name = full_data.classes
    num_classes = len(class_name)
    print(f'Classes Found: {class_name}, Train Size: {train_size}, Val Size: {val_size}')

    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V2)

    for param in model.features.parameters():
        param.requires_grad = False

    model.classifier[1] = nn.Linear(model.last_channel, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_acc = 0.0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / train_size
        epoch_acc = correct / total

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_top3 = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)

                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

                _, top3_pred = outputs.topk(3, 1, True, True)
                val_top3 += torch.eq(top3_pred, labels.view(-1, 1)).sum().item()
        
        val_epoch_loss = val_loss / val_size
        val_epoch_acc = val_correct / val_total
        val_epoch_top3 = val_top3 / val_total

        print(f'Epoch {epoch+1}/{epochs} | Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} | Val Loss: {val_epoch_loss:.4f} Acc: {val_epoch_acc:.4f} Top-3 Acc: {val_epoch_top3:.4f}')

        if val_epoch_acc > best_acc:
            best_acc = val_epoch_acc
            torch.save(model.state_dict(), 'gen3_model.pth')
            print(f'Model Saved ({best_acc:.4f})')

    print(f'Training Complete. Best Val Acc: {best_acc:.4f}')
if __name__ == '__main__':
    train_model()

Classes Found: ['Absol Pokemon', 'Aggron Pokemon', 'Altaria Pokemon', 'Anorith Pokemon', 'Armaldo Pokemon', 'Aron Pokemon', 'Azurill Pokemon', 'Bagon Pokemon', 'Baltoy Pokemon', 'Banette Pokemon', 'Barboach Pokemon', 'Beautifly Pokemon', 'Beldum Pokemon', 'Blaziken Pokemon', 'Breloom Pokemon', 'Cacnea Pokemon', 'Cacturne Pokemon', 'Camerupt Pokemon', 'Carvanha Pokemon', 'Cascoon Pokemon', 'Castform Pokemon', 'Chimecho Pokemon', 'Clamperl Pokemon', 'Claydol Pokemon', 'Combusken Pokemon', 'Corphish Pokemon', 'Cradily Pokemon', 'Crawdaunt Pokemon', 'Delcatty Pokemon', 'Deoxys Pokemon', 'Dusclops Pokemon', 'Duskull Pokemon', 'Dustox Pokemon', 'Electrike Pokemon', 'Exploud Pokemon', 'Feebas Pokemon', 'Flygon Pokemon', 'Gardevoir Pokemon', 'Glalie Pokemon', 'Gorebyss Pokemon', 'Groudon Pokemon', 'Grovyle Pokemon', 'Grumpig Pokemon', 'Gulpin Pokemon', 'Hariyama Pokemon', 'Huntail Pokemon', 'Illumise Pokemon', 'Jirachi Pokemon', 'Kecleon Pokemon', 'Kirlia Pokemon', 'Kyogre Pokemon', 'Lairon Po

In [6]:
pip install streamlit

   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.0 MB 235.4 kB/s eta 0:00:37
   -- ------------------------------------- 0.5/9.0 MB 235.4 kB/s eta 0:00:37
   -- ----------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
